In [4]:
import pandas
import numpy
import os

base_path = os.environ.get('DATA_PATH')
results_path = os.environ.get('RESULTS_PATH')
df = pandas.read_csv(os.path.join(base_path, 'LSE_YCA_prices.csv'), parse_dates=['Date'], dayfirst=True)
df.sort_values(by='Date', inplace=True)

print(df.head())

           Date   Open   High    Low   Close     Volume  Adjustment
1425 2018-07-04  194.0  194.0  194.0  194.00        NaN           1
1424 2018-07-05  200.0  204.0  193.0  204.00  1344894.0           1
1423 2018-07-06  200.0  204.0  200.0  204.00   968841.0           1
1422 2018-07-09  203.0  207.0  202.0  203.00   553271.0           1
1421 2018-07-10  203.0  204.0  200.0  201.25   963325.0           1


In [5]:
from indicators.average import rolling_average

from trading.trade import Trade
from trading.signal import cross_from_below

params = dict(short_ma = 6, long_ma = 13, ultra_long_ma = 26, stop = 0.1, take_profit = 0.05, holding_days = 10)


def strategy(params, dates, closes):
    short_ma = rolling_average(closes, params['short_ma'])
    long_ma = rolling_average(closes, params['long_ma'])
    ultra_long_ma = rolling_average(closes, params['ultra_long_ma'])
    
    crosses = cross_from_below(short_ma, long_ma)
    above_ultra_long = closes - ultra_long_ma
    
    live_deals = []
    dead_deals = []

    for i, (date, close, cross, above) in enumerate(zip( dates, closes, crosses, above_ultra_long)):
        if cross and above > 0:
            live_deals.append(Trade(date, close, 1, close*(1 - params['stop']), close*(1 + params['take_profit']), dates[min(i + params['holding_days'],len(dates)-1)] ))
        
        for d in live_deals:
            if close <= d.stop() or close >= d.take_profit() or date >= d.expiry():
                d.set_exit_price(close)
                dead_deals.append(d)
                live_deals.remove(d)
        
    for d in live_deals:
        d.set_exit_price(closes[-1])
        dead_deals += d

    return dead_deals
        
deals = strategy(params, df['Date'].values, df['Close'].values)
win_pct = sum([1 for d in deals if d.exit_price() - d.traded_price() > 0])/len(deals)
pandl= sum([d.exit_price() - d.traded_price() for d in deals])
losses = [d.exit_price() - d.traded_price() for d in deals if (d.exit_price() - d.traded_price()) <= 0] 
wins = [d.exit_price() - d.traded_price() for d in deals if (d.exit_price() - d.traded_price()) > 0]

p_and_l_ratio = (sum(wins)/len(wins))/(sum(numpy.abs(losses))/len(losses))

print(f'P&L {pandl:.2f}')
print(f'Deals {len(deals)}')
print(f"Win Pct {100*win_pct:.2f}")
print(f"Pts per deal {pandl/len(deals):.2f}")
print(f"P&L Ratio {p_and_l_ratio}")
print(f"Return {win_pct*(1+p_and_l_ratio)}")

P&L 149.70
Deals 41
Win Pct 58.54
Pts per deal 3.65
P&L Ratio 1.0877161598091827
Return 1.2220777520834238


In [6]:
deals_df = pandas.DataFrame.from_dict([Trade.to_dict(d) for d in deals])
deals_df.to_csv(os.path.join(results_path,'oil.csv'),mode='w')

In [7]:
from indicators.rsi import rsi

srs = numpy.array([  54.8, 56.8, 57.85, 59.85, 60.57, 61.1, 62.17, 60.60
        , 62.35, 62.15, 62.35, 61.45, 62.8, 61.37, 62.5, 62.57
        , 60.8, 59.37, 60.35, 62.35, 62.17, 62.55, 64.55
        , 64.37, 65.30, 64.42, 62.90, 61.60, 62.05, 60.05, 59.70
        , 60.90, 60.25, 58.27, 58.70, 57.72, 58.10, 58.20])

n = 14

rsi_srs = rsi(srs, n)

In [8]:
import unittests.trading.test_trailing_stop
import unittests.trading.test_cross_from_below

unittests.trading.test_trailing_stop.test_trailing_stop()
unittests.trading.test_cross_from_below.test_cross_from_below()






In [9]:
import plotly

plotly.plot(rsi_srs, kind='line')


In [10]:
from indicators.true_range import true_range, atr

opens = [52.8, 52.6, 52.0, 52.2, 52.10, 51.9, 51.5, 51.15, 51.50 ]
highs = [53, 52.75, 52.35, 52.45, 52.35, 52.1, 51.8, 51.6, 51.7 ]
lows = [52.5, 52.25, 51.85, 52.15, 51.75, 51.50, 51.00, 51.25, 51.40]
closes = [52.7, 52.55, 52.3, 52.4, 51.90, 51.65, 51.10, 51.55, 51.65 ]

print(len(opens), len(highs), len(lows), len(closes) )

tr = true_range(highs, lows, closes)

print(tr)

print(atr(highs, lows, closes, 7))


9 9 9 9
[0.5  0.5  0.7  0.3  0.65 0.6  0.8  0.5  0.3 ]
[       nan        nan        nan        nan        nan        nan
 0.57857143 0.56734694 0.52915452]
